In [149]:
import pandas as pd
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler
import pickle

In [148]:
### LOAD DATA
#Reading dataset
employee_df=pd.read_csv('C:/Users/lcast/Desktop/DS/GitHub/Employee-churn-predtion/Data/Human_Resources.csv')

## 3. DATA PREPROCESSING AND CLEANING

#### Converting into numerical
The attributes   'overtime' , 'Over18' along with the target 'Attritition'are boolean (Y/N) We replace by an integer (N/Y=> 0/1) before performing any visualization.

In [150]:
#employee_df['Attrition']=np.where(employee_df['Attrition']=='Yes',1,0)
employee_df['Attrition']=employee_df['Attrition'].apply(lambda x: 1 if x=='Yes' else 0)
employee_df['OverTime']=employee_df['OverTime'].apply(lambda x: 1 if x=='Yes' else 0)
employee_df['Over18']=employee_df['Over18'].apply(lambda x: 1 if x=='Y' else 0)

#### Removing irrelevant variables
The features 'EmployeeCount', 'Standardhours' and 'Over18'  do not change from one employee to the other, so can be dropped. It  also makes sense to drop 'EmployeeNumber' since is just and employee ID not relevant for the analysis.

In [151]:
employee_df.drop(['EmployeeCount','StandardHours','Over18','EmployeeNumber'],axis=1,inplace=True)

Since we are not complety sure of what this features represent (in a real situation we would ask to the HR department), and because in the dataset there is the variable Monthly Income that provides information about the salary, we remove all three.

In [152]:
employee_df.drop(['DailyRate','HourlyRate','MonthlyRate'],axis=1,inplace=True)

#### Split in train and test sets
Since the dataset is unbalanced, we do a stratified sampling to keep the same proportion of the two classes. 

In [153]:
split=StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42)
for train_index,test_index in split.split(employee_df,employee_df['Attrition']):
    employee_df_train=employee_df.loc[train_index]
    employee_df_test=employee_df.loc[test_index]

In [154]:
#Dividing between categorical and numerical features
Train_cat=employee_df_train[employee_df_train.columns[employee_df_train.dtypes==object]]
Train_num=employee_df_train[employee_df_train.columns[employee_df_train.dtypes!=object]]
Train_num=Train_num.drop('Attrition',axis=1)

#### Encoding

In [155]:
#Using OneHOtEncoding to encode the categorical variables
encoder=OneHotEncoder()
Train_cat_enc=encoder.fit_transform(Train_cat).toarray()
Train_cat_df=pd.DataFrame(Train_cat_enc,index=Train_cat.index)
Train_cat_df.columns=encoder.get_feature_names()

#### Scaling

In [156]:
#Scaling the numerical features
scaler=MinMaxScaler()
Train_num_df =pd.DataFrame(scaler.fit_transform(Train_num),columns=Train_num.columns,index=Train_num.index)

In [157]:
#Joining back the categorical and numerical features after the tretment.
Train_df_prepro=pd.concat([Train_cat_df,Train_num_df,pd.DataFrame(employee_df_train['Attrition'])],axis=1)

#### Preprocessing test data set

In [167]:
Test_cat=employee_df_test[employee_df_test.columns[employee_df_test.dtypes==object]]
Test_num=employee_df_test[employee_df_test.columns[employee_df_test.dtypes!=object]]
Test_num=Test_num.drop('Attrition',axis=1)
Test_cat_enc=encoder.transform(Test_cat).toarray()
Test_cat_df=pd.DataFrame(Test_cat_enc,index=Test_cat.index)
Test_cat_df.columns=encoder.get_feature_names()
Test_num_df =pd.DataFrame(scaler.transform(Test_num),columns=Test_num.columns,index=Test_num.index)

Test_df_prepro=pd.concat([Test_cat_df,Test_num_df,pd.DataFrame(employee_df_test['Attrition'])],axis=1)
#y_test=employee_df_test['Attrition']
#y_test=y_test.reset_index(drop=True)

In [169]:
file_path='C:/Users/lcast/Desktop/DS/GitHub/Employee-churn-predtion/Data/'

Train_df_prepro.to_pickle(file_path+ 'Train_df_prepro.pkl')
Test_df_prepro.to_pickle(file_path+ 'Test_df_prepro.pkl')